In [ ]:
import altair as alt
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns

from matplotlib import ticker

In [ ]:
alt.renderers.enable('notebook')

In [ ]:
import wrangler
import utils

In [ ]:
file = '10._Expenditure_-_DDO_Head_of_AccountSOE_and_VoucherBillNO_wise.csv'
filepath = utils.get_munged_filepath(file)
df = pd.read_csv(filepath, parse_dates=True)
df = wrangler.wrangle_data_for_consolidated_query(df, ['DDODESC', 'DISTRICT', 'TREASURY', 'DDO'])

In [ ]:
def plot_district_ranking_for_category(category):
    '''
    Plots bar chart for the Total amount spent in the given category for all districts to see where each district stands.
    for example, we can rank the districts based on their Total GROSS distribution.
    
    input- category:GROSS
    '''
    data = df.groupby('DISTRICT').sum().dropna()

    return alt.Chart(data.reset_index(), width=600, height=300).mark_bar().encode(
        x=alt.X('{}:Q'.format(category),
                axis=alt.Axis(title='Amount (in Rupee)', format='s')),
        y=alt.Y('DISTRICT:O',
                sort=alt.EncodingSortField(field=category, op='sum', order='descending'),
                axis=alt.Axis(title='Districts')),
        color='DISTRICT').properties(
        title='District wise Expenditure: {}'.format(category),
    )

In [ ]:
def compare_categories_for_districts(categories):
    '''
    Compare Total expense made in different categories for each district,
    for example we can compare the distribution of Total GROSS amount and NETPAYMENT for all districts.
    
    input - categories:['GROSS', 'NETPAYMENT']
    '''
    district_grouped = df.groupby(['DISTRICT']).sum().dropna().reset_index()
    data = district_grouped.melt(id_vars='DISTRICT', value_vars=categories,
                     var_name='Type', value_name='Amount')
    
    #NOTE: need to make it work.
    #bar_width = 100/(0.6*len(categories)-4)
    
    return alt.Chart(data, height=350, width=52).mark_bar().encode(
        
        column=alt.Column('DISTRICT',
                          sort=alt.EncodingSortField(field='Amount', op='sum', order='descending')),

        x=alt.X('Type:N',
                axis=alt.Axis(title=None), sort=categories),

        y=alt.Y('Amount:Q',
                axis=alt.Axis(title='Amount (in Rupee)', format='s', grid=False)),
                
        color=alt.Color('Type', scale=alt.Scale(range=['#1d4d74', '#3c7d8f', '#9bc2b6', '#6fa6a4']))
        
    ).properties(
        title='Compare District wise Expenditure: %s' % ', '.join(categories),
    )

In [ ]:
def plot_pn_expenditure(category):
    '''
    Plot the percentage expense on Plan and Non Plan expenditures.
    
    input - category:'GROSS'
    '''
    data = df.groupby(['DISTRICT', 'PN']).sum().dropna().reset_index()
    data = data[data.PN != '']
    
    return alt.Chart(data, height=400, width=52).mark_bar().encode(

        column=alt.Column('DISTRICT',
                          sort=alt.EncodingSortField(field=category, op='sum', order='descending')),

        y=alt.Y('{}:Q'.format(category),
                axis=alt.Axis(title='Amount (in Rupee)', format='s', grid=False),
                ),
                
        color=alt.Color('PN', scale=alt.Scale(range=['#a0d9ef', '#20a7db'])),
        order=alt.Order('PN', sort='ascending')

    ).properties(
        title='Compare District wise Expenditure: {}'.format(category),

    ).transform_calculate(
        'PN', alt.expr.if_(alt.datum.PN == 'P', 'Plan', 'Non-plan')
    )

In [ ]:
def plot_time_series_for_district_category(district, category):
    '''
    Plot a feature across a monthly timeline for a district.
    For example we could see how GROSS amount was distributed from April 2017 to August 2018 for Shimla.
    
    input - district:'SHIMLA', category:'GROSS'
    '''
    # group data by districts and date.
    district_grouped = df.groupby(['DISTRICT', 'DATE']).sum()

    # get monthly data for input district
    monthly_data = district_grouped.loc[district].resample('M').sum()
    
    # select input feature from district data.
    data = monthly_data[category]
    
    # create a selection for mouseover on fields(here Date).
    peak = alt.selection_single(on='mouseover', nearest=True, empty='none')
    
    # create the main line chart.
    line = alt.Chart().mark_line().encode(
        
        x=alt.X('DATE:T'.format(category),
                axis=alt.Axis(title='Expenditure Timeline (Monthly)', format='%b %Y')),
        
        y=alt.Y('{}:Q'.format(category),
                axis=alt.Axis(title='Amount (in Rupees)', format='s')),
    )
    
    selectors = alt.Chart().mark_point().encode(
        x='DATE:T',
        opacity=alt.value(0),
    ).add_selection(
        peak
    )
    
    # Draw points on the line, and highlight based on selection
    points = line.mark_point().encode(
        opacity=alt.condition(peak, alt.value(1), alt.value(0))
    )
    
    # Draw text labels near the points, and highlight based on selection
    text = line.mark_text(align='left', dx=5, dy=-5).encode(
        text=alt.condition(peak, '{}:Q'.format(category), alt.value(' '), format='.3s'),
    )
    
    # Put the five layers into a chart and bind the data
    return alt.layer(line, selectors, points, text,
                     data=data.reset_index(), 
                     width=700, height=350,
                     title='{} Monthly Expenditure: {}'.format(district, category))

In [ ]:
def plot_major_head_wise_expenditure_for_district(major_heads, district, category):
    '''
    Plot input Major head wise expenditure for asked district and category.
    
    input - major_heads: :list of major heads:, district:'SHIMLA', category:'GROSS'
    '''
    # group data by major head and districts
    district_maj_grouped = df.groupby(['DISTRICT', 'MAJ']).sum().dropna()
    
    # select data for input district and drop the empty MAJ row.
    required_district = district_maj_grouped.loc[district][1:]
    
    # select only asked major heads
    data = required_district.loc[major_heads]
    
    colors = ['golden yellow', 'flat blue', 'pale brown', 'grey purple', 'dusty orange',
          'cool grey', 'pale red', 'windows blue', 'brownish orange', 'dark seafoam green']
    
    sort_order = alt.EncodingSortField(field=category, op='sum', order='descending')
    return alt.Chart(
        
        data=data.reset_index(),
        width=500, height=300,
        title='Major Head wise Expenditure for {}: {}'.format(district, category)
        
    ).mark_bar().encode(
        
        x=alt.X('{}:Q'.format(category),
                axis=alt.Axis(title='Amount (in Rupee)', format='s', grid=False)),  
        
        y=alt.Y('MAJ:O',
                sort=sort_order,
                axis=alt.Axis(title='Major Heads')),
        
        color='MAJ'
    )

In [ ]:
def plot_top_10_major_heads_for_district(district, category):
    '''
    Plot Top 10 Major heads of expenditure for asked district and category.
    
    input - district:'SHIMLA', category:'GROSS'
    '''
    # group data by major head and districts
    district_maj_grouped = df.groupby(['DISTRICT', 'MAJ']).sum().dropna()
    
    # select data for input district and drop the empty MAJ row.
    required_district = district_maj_grouped.loc[district][1:]
    
    # sort it descending and get only top 10.
    data = required_district.sort_values(by=category, ascending=False)[:10]
    
    return alt.Chart(
        
        data=data.reset_index(),
        width=500, height=300,
        title='Top 10 Major Heads of Expenditure for {}: {}'.format(district, category)
        
    ).mark_bar().encode(
        
        x=alt.X('{}:Q'.format(category),
                axis=alt.Axis(title='Amount (in Rupee)', format='s', grid=False)),  
        
        y=alt.Y('MAJ:O',
                sort=alt.EncodingSortField(field=category, op='sum', order='descending'),
                axis=alt.Axis(title='Major Heads')),
        
        color='MAJ'
    )

# adding text markers, mess up the sort order, a bug in vega.
#     # Draw text labels near the points, and highlight based on selection
#     text = bar.mark_text(align='left', dx=5, dy=3).encode(
#         text=alt.Text('{}:Q'.format(category), format='.3s'),
#     )
    
#     # Put the layers into a chart and bind the data
#     return alt.layer(bar, text,
#                      data=data.reset_index(), 
#                      width=500, height=300,
#                      title='Major Head wise Expenditure for {}: {}'.format(district, category))

In [ ]:
def plot_amount_distribution():
    '''
    Plot district wise distribution of sanctioned amount.
    The amount is distributed in three categories: AGDED, BTDED and NETPAYMENT,
    so we can analyse how does this distribution vary across districts.
    '''
    categories = ['NETPAYMENT', 'AGDED', 'BTDED']
    district_grouped = df.groupby('DISTRICT').sum().dropna().reset_index()

    data = district_grouped.melt(id_vars='DISTRICT', value_vars=categories,
                     var_name='Type', value_name='Amount')
    
    return alt.Chart(data, height=300, width=45).mark_bar().encode(
        
        column=alt.Column('DISTRICT'),

        y=alt.Y('sum(Amount):Q',
                axis=alt.Axis(title='Amount (in Rupee)', format='s')),
   
        color=alt.Color('Type', scale=alt.Scale(range=['#8a2c02', '#d75404', '#f08b33']))
        
    ).properties(
        title='District wise distribution of Amount ',
    )